In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


把train.csv給輸入並轉成陣列

In [8]:
def createl(train_data):
    save=['Pclass','Sex','Fare','Age']
    for s in save:
        print(s,train_data[s].isna().sum())
    train_data['Age']=train_data['Age'].fillna(0)
    l=[]
    serv_dict={1:0.6,2:0.44,3:0.26}
    s1=train_data['Pclass'].tolist()
    s2=[serv_dict[x] for x in s1]
    l.append(s2)
    s1=train_data['Sex'].tolist()
    s2=[1 if x=='male' else -1 for x in s1]
    l.append(s2)
    s1=train_data['Fare']
    s2=[math.log(x+1e-10) for x in s1]
    l.append(s2)
    s1=train_data['Age'].tolist()
    s2=[x==1 if x<=15 else -1 for x in s1]
    l.append(s2)
    return l

In [9]:
train_data=pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.info()
l=createl(train_data)
s1=train_data['Survived'].tolist()
s2=[1 if x==1 else -1 for x in s1]
l.append(s2)
data=np.array(l,ndmin=3)
data=data.transpose()
np.random.shuffle(data)
train_data=data[0:800,:,:]
val_data=data[801:,:,:]
print(train_data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
Pclass 0
Sex 0
Fare 0
Age 177
(800, 5, 1)


數據預處理完。
$Sex: \in [-1,1]  \\
Pclass\in [1,2,3] \\
Age \in [1,2,3]  \\
ans \in [1,0]$

In [10]:
def sign(x):
    if x>=0:
        return 1
    return -1

In [11]:
def Count_perform(tdata,Size,w,n):
    error=0
    for index in range(0,Size):
        ans_i=data[index][n]
        data_i=data[index][0:n]
        k=sign(w@(data_i))
        if k!=ans_i:
            error+=1
    return error

In [12]:
n=train_data.shape[1]-1
w=np.zeros((1,n))
Size=train_data.shape[0]
min_error=Size+1
print(Size)
ans_w=w
indexs=[i for i in range(0,Size)]
for i in range(100000):
    random.shuffle(indexs)
    for index in indexs:
        ans_i=train_data[index][n]
        data_i=train_data[index][0:n]
        if sign(w@(data_i))!=ans_i:
            w+=(ans_i@(data_i.T))
    E=Count_perform(val_data,val_data.shape[0],w,n)
    if E<min_error:
        min_error=E
        ans_w=w
print(min_error)
print(ans_w)

800
16
[[-0.18       -3.          0.23418326 -1.        ]]


In [13]:
test_data=pd.read_csv('/kaggle/input/titanic/test.csv')
l=createl(test_data)
test=np.array(l,ndmin=3)
test=(test.transpose())
Survived=[]
for i in range(test.shape[0]):
    Sign=sign(ans_w@test[i])
    Survived.append((Sign+1)//2)
term={"PassengerId":test_data['PassengerId'].tolist(),
        "Survived":Survived}
pd_term=pd.DataFrame(term)
pd_term.to_csv('./ans.csv',index=False)
print('done')

Pclass 0
Sex 0
Fare 1
Age 86
done
